Progress Report 1
----

**Team Members**

Yaqian Cheng, Department of Statistical Science

Mengrun Li, Department of Statistical Science

**Github repository**

<https://github.com/cici7941/Sta_663_Statistical_Computation_Final_Project>

**Choice of paper** 

*Scalable K-Means++*

**Abstract**

*K-means* is one of the most popular clustering methods. A good initialization of *k-means* is essential for obtaining the global optimal solution and efficiency. However, there are two main obstacles with traditional *k-means* method. One is theoretical inefficiency and the other one is that its final solution is locally optimal. A better algorithm, *k-means++* addresses the second problem with an improved initialization procedure of the cluster centers. But this *k-means++* initialization is not parallelizable, because the selection for the *i*th center depends on the previous *i-1* centers [1]. Therefore, *k-means||*, a parallelizable version of *k-means++*, has been raised, which can both improve the final solution and run faster. In this report, we implemented the algorithm in the paper "Scalable K-Means++" in Python, compared the clustering cost and runtime between *k-means*, *k-means++* and *k-means||*, performed tests for main functions, profiled the performance of the algorithm and identified bottlenecks, and performed optimization using Cython. We then apply *k-means||* to a massive dataset to evaluate its performance.

**Outline**

1. Introduction
2. Algorithm  
    2.1 K-Means  
    2.2 K-Means++  
    2.3 K-Means||  
3. Code Testing
4. Profiling and Optimization
5. Application and Comparison

In [5]:
import scipy.linalg as la
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# helper functions
def euc_dist(x, y):
    return la.norm(x-y)

def centroid(X):
    return X.mean(0)
    
def d(x, Y):
    minDist = float("inf")
    for yi in Y:
        dist = euc_dist(x, yi)
        if(dist < minDist):
            minDist = dist
    return minDist

def cost(Y, C):
    cost = 0
    for yi in Y:
        cost += d(yi, C)**2
    return cost

In [3]:
# K-Means++
def kmeansPlus(data, k):
    idx = np.random.choice(data.shape[0], 1)
    C = data[idx, :]
    while(C.shape[0] < k):
        prob = [d(xi,C)**2 for xi in data]/cost(data,C)
        new = data[np.random.choice(data.shape[0], size=1, p=prob),:]
        if(new.tolist() not in C.tolist()):
            C = np.r_[C, new]
    return C

**Initialization algorithm: k-means||**  
*k-means||* uses an oversampling factor *l* = $\Omega$(k), which is unlike *k-means++*. Intuitively, *l* should be thought of as $\Theta$(k). This initialization algorithm picks an initial center uniformly from the dataset and computes $\psi$, here is initial cost of the clustering of this selection. Then do log$\psi$ iterations and in each iteration, it samples each x with probability *l*$d^2(x,C)/\psi_X(C)$ given current set C of centers. If the point is sampled, it will be added to C and the quantity $\phi_X(C)$ updated and interation continued.

***Algorithm*** *k-means||*(k,l)initialization  
1. C <- sample a point uniformly at random from X  
2. $\psi$ = $\phi_X(C)$  
3. for O(log$\psi$) times do  
      C' <- sample each point x $\in$ X independently with probability $p_x = \frac{l*d^2(x,C)}{\phi_X(C)}$  
      C <- C $\cup$ C'  
   end for  
4. For x $\in$ C, set $w_x$ to be the number of points in X closer to x than any other point in C  
5. Recluster the weighted points in C into k clusters

[8]
first centroid: [[  9.   11.5]]
initial cost: 690.99


In [96]:
# K-Means||
X_ori = np.array([[1,0],[1.5,0],[3,5],[3,4.8],[3.6,4.5],[4,4.9],[9,12],[9,11.5],[8.7,11.7]])
X = X_ori
k = 3
l = 2
##step1
##Sample a point uniformly at random from X
idx1 = np.random.choice(X_ori.shape[0],1,replace = False)
C = X[idx1-1,:]
print(idx1)
print("first centroid:",C)
##step2
##initial cost
gamma1 = cost(X,C)
print("initial cost:",gamma1)
##remove first centroid
X = np.delete(X, idx1-1, axis=0)
X
plt.plot(X[:,0],X[:,1],"o")

In [97]:
##step3
for i in range(int(round(np.log(gamma1)))):
    Ct = []
    idx = []
    gamma = cost(X,C)
    prob = [l*d(x,C)**2/gamma for x in X]
    #new = data[np.random.choice(X.shape[0], size=1, p=prob),:]
    for j in range(len(X)):
        point = np.random.binomial(1,prob[j],size = 1)
        if point == 1:
           Ct.append(X[j,:])
           idx.append(j)
    X = np.delete(X, idx, axis = 0)
    C = np.concatenate((C,np.array(Ct).reshape(len(Ct),2)),axis = 0)
C

ValueError: p > 1

In [83]:
gamma = cost(X,C)
prob = [l*d(x,C)**2/gamma for x in X]

In [84]:
prob

[0.5603901177358084,
 0.5395101974836329,
 0.2290055769593448,
 0.23665705741304527,
 0.23011019209526634,
 0.20316835951181406,
 0.0006735458145863082,
 0.0004849529865021442]

In [85]:
[d(x,C)**2/gamma for x in X]

[0.2801950588679042,
 0.26975509874181647,
 0.1145027884796724,
 0.11832852870652263,
 0.11505509604763317,
 0.10158417975590703,
 0.0003367729072931541,
 0.0002424764932510721]

In [86]:
Ct = []
idx = []
gamma = cost(X,C)
prob = [l*d(x,C)**2/gamma for x in X]
#new = data[np.random.choice(X.shape[0], size=1, p=prob),:]
prob

[0.5603901177358084,
 0.5395101974836329,
 0.2290055769593448,
 0.23665705741304527,
 0.23011019209526634,
 0.20316835951181406,
 0.0006735458145863082,
 0.0004849529865021442]

In [87]:
for j in range(len(X)):
    point = np.random.binomial(1,prob[j],size = 1)
    if point == 1:
       Ct.append(X[j,:])
       idx.append(j)
X = np.delete(X, idx, axis = 0)
C = np.concatenate((C,np.array(Ct).reshape(len(Ct),2)),axis = 0)
C

array([[  9. ,  12. ],
       [  1.5,   0. ]])

In [88]:
Ct

[array([ 1.5,  0. ])]

In [89]:
Ct = []
idx = []
gamma = cost(X,C)
prob = [l*d(x,C)**2/gamma for x in X]
#new = data[np.random.choice(X.shape[0], size=1, p=prob),:]
prob

[0.004623636027371924,
 0.5039763269835398,
 0.4677270205289438,
 0.45607545773996666,
 0.5596449047530978,
 0.004623636027371924,
 0.0033290179397078012]

In [90]:
for j in range(len(X)):
    point = np.random.binomial(1,prob[j],size = 1)
    if point == 1:
       Ct.append(X[j,:])
       idx.append(j)
X = np.delete(X, idx, axis = 0)
C = np.concatenate((C,np.array(Ct).reshape(len(Ct),2)),axis = 0)
C

array([[  9. ,  12. ],
       [  1.5,   0. ],
       [  3.6,   4.5]])

In [91]:
Ct

[array([ 3.6,  4.5])]

In [92]:
Ct = []
idx = []
gamma = cost(X,C)
prob = [l*d(x,C)**2/gamma for x in X]
#new = data[np.random.choice(X.shape[0], size=1, p=prob),:]
prob

[0.24271844660194164,
 0.5922330097087377,
 0.43689320388349495,
 0.3106796116504854,
 0.24271844660194164,
 0.1747572815533988]

In [93]:
for j in range(len(X)):
    point = np.random.binomial(1,prob[j],size = 1)
    if point == 1:
       Ct.append(X[j,:])
       idx.append(j)
X = np.delete(X, idx, axis = 0)
C = np.concatenate((C,np.array(Ct).reshape(len(Ct),2)),axis = 0)
C

array([[  9. ,  12. ],
       [  1.5,   0. ],
       [  3.6,   4.5],
       [  1. ,   0. ],
       [  3. ,   5. ],
       [  9. ,  11.5]])

In [94]:
X

array([[  3. ,   4.8],
       [  4. ,   4.9],
       [  8.7,  11.7]])

In [95]:
Ct = []
idx = []
gamma = cost(X,C)
prob = [l*d(x,C)**2/gamma for x in X]
#new = data[np.random.choice(X.shape[0], size=1, p=prob),:]
prob

[0.16326530612244916, 1.3061224489795915, 0.5306122448979594]